### Import packages 

In [1]:
import sqlite3
import pandas as pd

### Connect to database

In [39]:
conn = sqlite3.connect('amazon_db.db')
c = conn.cursor()

In [40]:
c.execute("""SELECT * FROM books
LIMIT 5; """) 

In [41]:
df = pd.DataFrame(c.fetchall())
df.columns = [i[0] for i in c.description]
df

,books,num_copies
0,A Elephant Ate Under A Fish,4226324
1,A Boy Jumped Outside A Lake,4048899
2,One Man Hopped Under The Tree,774329
3,A Person Threw Behind The Notebook,1359606
4,The Tree Ran Through A Mountain,4703380


In [42]:
c.execute("""SELECT * FROM author
LIMIT 5; """) 

In [43]:
df = pd.DataFrame(c.fetchall())
df.columns = [i[0] for i in c.description]
df

,author,book
0,David Taylor,A Guy Dove Outside The Desk
1,Sidney Osborne,A Fish Hopped Through A Lake
2,Robert Martin,The Woman Winced Outside A Desk
3,Greg Simpson,A Banana Winced Outside The Banana
4,Edgar Williamson,A Person Ate Behind The Potato


### Questions

1. How many rows are in the author and books tables?

2. How many authors (share) didn't sell some of their book? 

3. How many author name duplicates are there in the author table?   

4. How many authors (share) didn't sell any book?

5. Return the names of these authors.

6. How many books in total each author has sold? 

7. Choose only authors who have sold more than 400K copies
    - use subquery first
    - can you use something else? 

8. Get all authors that have their name starting with 'a'
    - Use substr first
    - Can you use a wild card? 

9. Count the number of authors that have their name starting with 'a' and that have single books that
    sold more than 200K copis
    - Can you use WITH

### 1.

In [44]:
c.execute("""SELECT count(*) AS num_rows FROM author; """) 

In [45]:
df = pd.DataFrame(c.fetchall())
df.columns = [i[0] for i in c.description]
df

,num_rows
0,100000


In [46]:
c.execute("""SELECT count(DISTINCT(author)) AS distinct_author, count(author) as total_authors FROM author; """) 

In [48]:
c.execute("""SELECT count(*) num_rows FROM books; """) 

In [49]:
df = pd.DataFrame(c.fetchall())
df.columns = [i[0] for i in c.description]
df

,num_rows
0,80000


### 2.

In [69]:
c.execute("""SELECT count(DISTINCT(author)) authors_didnt_sell FROM author
LEFT JOIN books ON books.books = author.book
WHERE books IS NULL ; """) 

In [70]:
df = pd.DataFrame(c.fetchall())
df.columns = [i[0] for i in c.description]
df

,authors_didnt_sell
0,19545


# 3. 

In [82]:
c.execute("""SELECT count(DISTINCT(author)) as dist_author, COUNT(author) as total FROM author ; """) 

In [83]:
df = pd.DataFrame(c.fetchall())
df.columns = [i[0] for i in c.description]
df

,dist_author,total
0,92557,100000


# 4 & 5

In [92]:
c.execute("""SELECT count(*) FROM (SELECT author, SUM(COALESCE(num_copies, 0)) authors_didnt_sell FROM author
LEFT JOIN books ON books.books = author.book
GROUP BY author
HAVING SUM(COALESCE(num_copies, 0)) = 0 ) AS bad_authors; """) 

In [93]:
df = pd.DataFrame(c.fetchall())
df.columns = [i[0] for i in c.description]
df

,count(*)
0,17724


In [94]:
## REmove outerquery to get number 5

# 6. 

In [97]:
c.execute("""SELECT author, SUM(COALESCE(num_copies, 0)) total_sales FROM author
LEFT JOIN books ON books.books = author.book
GROUP BY author; """) 

In [98]:
df = pd.DataFrame(c.fetchall())
df.columns = [i[0] for i in c.description]
df

,author,total_sales
0,Aaron Agrela,4277967
1,Aaron Allen,0
2,Aaron Allman,2840107
3,Aaron Anderson,4306689
4,Aaron Baez,1039486
5,Aaron Bean,3475616
6,Aaron Becker,3630984
7,Aaron Blain,0
8,Aaron Blair,4479231
9,Aaron Blanch,2318165


# 7.

### a

In [100]:
c.execute("""SELECT * FROM

(SELECT author, SUM(num_copies) total_sales FROM author
LEFT JOIN books ON books.books = author.book
GROUP BY author)

WHERE total_sales > 400000
ORDER BY total_sales DESC; """) 

In [101]:
df = pd.DataFrame(c.fetchall())
df.columns = [i[0] for i in c.description]
df

,author,total_sales
0,Robert Williams,62278835
1,Robert Johnson,45011634
2,John Smith,42209406
3,William Smith,41262268
4,John Williams,41098358
5,Mary Williams,40686709
6,John Brown,40564334
7,James Williams,38274989
8,John Johnson,37746156
9,Michael Smith,37141959


### b

In [102]:
c.execute("""SELECT author, SUM(num_copies) total_sales FROM author
LEFT JOIN books ON books.books = author.book
GROUP BY author
HAVING SUM(num_copies) > 400000
ORDER BY total_sales DESC; """) 

In [103]:
df = pd.DataFrame(c.fetchall())
df.columns = [i[0] for i in c.description]
df

,author,total_sales
0,Robert Williams,62278835
1,Robert Johnson,45011634
2,John Smith,42209406
3,William Smith,41262268
4,John Williams,41098358
5,Mary Williams,40686709
6,John Brown,40564334
7,James Williams,38274989
8,John Johnson,37746156
9,Michael Smith,37141959


### 8.

### a

In [104]:
c.execute("""SELECT author FROM author
WHERE SUBSTR(author, 0, 2) == 'A'; """) 

In [105]:
df = pd.DataFrame(c.fetchall())
df.columns = [i[0] for i in c.description]
df

,author
0,Anthony Blache
1,Anthony Cobham
2,Aaron Latham
3,Anton Hall
4,Amber Powers
5,Ann Duong
6,Andres Miller
7,Ashley Palmer
8,Alice Wagner
9,Antwan Pereira


### b

In [109]:
c.execute("""SELECT author FROM author
WHERE author LIKE  'A%'; """) 

In [110]:
df = pd.DataFrame(c.fetchall())
df.columns = [i[0] for i in c.description]
df

,author
0,Anthony Blache
1,Anthony Cobham
2,Aaron Latham
3,Anton Hall
4,Amber Powers
5,Ann Duong
6,Andres Miller
7,Ashley Palmer
8,Alice Wagner
9,Antwan Pereira


### 9.

In [129]:
### With subquery

In [127]:
c.execute("""SELECT * FROM 

(SELECT author, COALESCE(num_copies, 0) AS sold FROM author
LEFT JOIN books ON books.books = author.book) AS author_books

WHERE sold > 2000000
AND author LIKE 'A%';""")

In [128]:
df = pd.DataFrame(c.fetchall())
df.columns = [i[0] for i in c.description]
df

,author,sold
0,Anthony Blache,1425062
1,Anthony Cobham,2011172
2,Aaron Latham,354546
3,Ann Duong,708624
4,Andres Miller,2910141
5,Ashley Palmer,3543169
6,Alice Wagner,4364647
7,Antwan Pereira,827510
8,Antone Morgan,1136448
9,Anita Mix,1108072


In [132]:
###  With 'WITH' statement

In [130]:
c.execute(""" 
WITH author_books AS
(SELECT author, COALESCE(num_copies, 0) AS sold FROM author
LEFT JOIN books ON books.books = author.book) 
SELECT * FROM author_books
WHERE sold > 2000000
AND author LIKE 'A%';""")

In [131]:
df = pd.DataFrame(c.fetchall())
df.columns = [i[0] for i in c.description]
df

,author,sold
0,Anthony Blache,1425062
1,Anthony Cobham,2011172
2,Aaron Latham,354546
3,Ann Duong,708624
4,Andres Miller,2910141
5,Ashley Palmer,3543169
6,Alice Wagner,4364647
7,Antwan Pereira,827510
8,Antone Morgan,1136448
9,Anita Mix,1108072
